In [65]:
import pandas as pd
import numpy as np
import turicreate as tc

import numpy as np
import scipy
import math
import random

import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds


In [2]:
following = pd.read_csv('following.csv')
post = pd.read_csv('posts (1).csv')
users = pd.read_csv('users.csv')
thoughts = pd.read_csv('thoughts.csv')
contact_settings = pd.read_csv('contact_settings.csv')
ext_feed = pd.read_csv('extfeeds.csv')

In [3]:
#users.head()

In [4]:
#following.head()

In [53]:
post.tail(5)

,user_id,title,content,tags,slug,created_at,updated_at,image,status_id,action,post_id,actionStrength
871,45,jbhjkhkbb,jkhbjilbjhb v hvhjbv h hbv jhkv,NaN,jbhjkhkbb-748,2019-09-05 16:38:18,2019-09-05 16:38:18,NaN,NaN,publish,NaN,3
872,3,Testing tags and really long posts,<p>Testing tags and really long posts.</p>\n<p...,"Politics, Sports",testing-tags-and-really-long-posts-38a,2019-09-06 14:39:26,2019-09-06 14:39:26,NaN,NaN,no_action,NaN,1
873,3,Testing image,"<p>Test if image works</p>\n<p><img src=""/stor...",NaN,testing-image-5f7,2019-09-06 14:39:26,2019-09-06 14:39:26,jeff.ogah/images/img-07flurib94.png,NaN,no_action,NaN,1
874,3,Test,"<p><img src=""/storage/jeff.ogah/images/img-x89...",NaN,test-03f,2019-09-06 14:39:26,2019-09-06 14:39:26,jeff.ogah/images/img-x89tf05exw.jpeg,NaN,no_action,NaN,1
875,3,My first post,<p>About time I made one.</p>\n<p><strong>Bold...,NaN,my-first-post-10e,2019-09-06 14:39:26,2019-09-06 14:39:26,NaN,NaN,no_action,NaN,1


In [6]:
post['action'].value_counts()

publish    4
draft      1
Name: action, dtype: int64

In [7]:
post['action'] = post['action'].replace(np.nan, 'no_action')

In [32]:
action_type = {
    'no_action':1,
    'draft': 2,
    'publish': 3
}

In [33]:
post['actionStrength'] = post['action'].apply(lambda x: action_type[x])

In [34]:
post_group = post.groupby(['user_id']).agg({'actionStrength': 'count'}).reset_index()

#articles_grouped = merged.groupby(['contentId']).agg({'eventStrength': 'count'}).reset_index()

In [35]:
grouped_sum = post_group['actionStrength'].sum() # Get the sum of all counts
post_group['percentage']  = post_group['actionStrength'].div(grouped_sum)*100 # Score each articles
post_group.sort_values(['actionStrength', 'user_id'], ascending = [0,1]) ; post_group.head()

,user_id,actionStrength,percentage
0,1,4,0.456621
1,2,12,1.369863
2,3,15,1.712329
3,4,2,0.228311
4,5,1,0.114155


In [36]:
users = post['user_id'].unique() ; print("# of users" , len(users))

# of users 308


In [37]:
items = post['title'].unique() ; print("# of items" , len(items))

# of items 658


In [38]:
#df = post[['user_id', 'title', 'content', 'action_type', '']]

In [39]:
train_data, test_data = train_test_split(post, test_size = 0.20, random_state=0)

In [40]:
#is_model = Rd.item_similarity_recommender_py()
#is_model.create(train_data, 'personId', 'contentId')

In [41]:
post.head()

,user_id,title,content,tags,slug,created_at,updated_at,image,status_id,action,post_id,actionStrength
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,NaN,what-i-have-learnt-so-far-on-html-985,2019-08-25 19:39:01,2019-08-28 11:30:00,1,NaN,no_action,NaN,1
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6,2019-08-25 19:44:36,2019-08-28 11:30:00,1,NaN,no_action,NaN,1
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,NaN,my-laziness-in-the-open-029,2019-08-25 19:50:33,2019-08-28 11:30:00,1,NaN,no_action,NaN,1
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,NaN,my-task-2-649,2019-08-25 19:51:36,2019-08-28 11:30:00,1,NaN,no_action,NaN,1
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str...",NaN,task-2-7e5,2019-08-25 19:51:39,2019-08-28 11:30:00,1,NaN,no_action,NaN,1


In [42]:
train_data.head()

,user_id,title,content,tags,slug,created_at,updated_at,image,status_id,action,post_id,actionStrength
45,2168,StartNG Stage 1,"Hi guys, my name is Damian Tochukwu, i go by t...",NaN,startng-stage-1-cad,2019-08-25 22:47:25,2019-08-28 11:30:03,NaN,NaN,no_action,NaN,1
172,1089,My Start.Ng Journey,<h1>The Beginning</h1>\n<p>I have always wante...,NaN,my-startng-journey-c81,2019-08-26 13:54:07,2019-08-28 11:30:25,NaN,NaN,no_action,NaN,1
492,93,My HTML Task,"<p><img src=""/storage/eni.fanegan/images/img-q...",NaN,my-html-task-f7c,2019-08-27 09:38:22,2019-08-28 11:30:28,eni.fanegan/images/img-v7fjystlk7.png,NaN,no_action,NaN,1
820,2296,Getting Started with HTML,Help everyone...\n\nToday we will be looking a...,NaN,getting-started-with-html-6ae,2019-08-27 17:43:39,2019-08-28 11:30:47,1,NaN,no_action,NaN,1
651,1201,My First Language!!,"<p>Hello Lucid, I've learnt so much on <strong...",NaN,my-first-language-d74,2019-08-27 13:13:45,2019-08-28 11:30:37,benjamin.bala.91/images/img-h0zt9aj93f.png,NaN,no_action,NaN,1


In [43]:
col = ['tags', 'slug', 'created_at', 'updated_at', 'image', 'status_id', 'action', 'post_id']

train_data = train_data.drop(train_data[col], axis=1)

In [44]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r",", "")
    df[text_field] = df[text_field].str.replace(r"!", "")
    df[text_field] = df[text_field].str.replace(r"(", "")
    df[text_field] = df[text_field].str.replace(r")", "")
    df[text_field] = df[text_field].str.replace(r"?", "")
    df[text_field] = df[text_field].str.replace(r"*", "")
    df[text_field] = df[text_field].str.replace(r"$", "")
    df[text_field] = df[text_field].str.replace(r"%", "")
    df[text_field] = df[text_field].str.replace(r"&", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

train_data = standardize_text(train_data, "content")
test_data = standardize_text(test_data, 'content')

#cleaned_data.to_csv("clean_data.csv")
train_data.head()

/home/godwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/godwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/godwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

,user_id,title,content,actionStrength
45,2168,StartNG Stage 1,hi guys my name is damian tochukwu i go by the...,1
172,1089,My Start.Ng Journey,h1 the beginning h1 \n p i have always wante...,1
492,93,My HTML Task,"p img src "" storage eni fanegan images img q...",1
820,2296,Getting Started with HTML,help everyone \n\ntoday we will be looking a...,1
651,1201,My First Language!!,p hello lucid i've learnt so much on strong ...,1


In [45]:
train_data

,user_id,title,content,actionStrength
45,2168,StartNG Stage 1,hi guys my name is damian tochukwu i go by the...,1
172,1089,My Start.Ng Journey,h1 the beginning h1 \n p i have always wante...,1
492,93,My HTML Task,"p img src "" storage eni fanegan images img q...",1
820,2296,Getting Started with HTML,help everyone \n\ntoday we will be looking a...,1
651,1201,My First Language!!,p hello lucid i've learnt so much on strong ...,1
...,...,...,...,...
835,1717,My html project,p html is one of the many languages used for ...,1
192,280,My experience and what I have learnt so far.,p learning is not attained by chance it must...,1
629,455,HTML among other things,p although i've already been using html for a...,1
559,952,curriculum vitae,p okey guys here is my pure html work without...,1


In [46]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 45 to 684
Data columns (total 4 columns):
user_id           700 non-null int64
title             700 non-null object
content           700 non-null object
actionStrength    700 non-null int64
dtypes: int64(2), object(2)
memory usage: 27.3+ KB


In [47]:
train_data_sframe = tc.SFrame(train_data)


In [70]:
item_sim_model = tc.item_similarity_recommender.create(train_data_sframe, user_id='user_id', 
                                                       item_id='content', 
                                                       target='actionStrength', similarity_type='cosine')

Warning: Ignoring columns title;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 700 observations with 285 users and 608 items.

Data prepared in: 0.011298s

...

| 93.123ms                            | 100              | 608             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.217499s

In [62]:
item_sim_recomm = item_sim_model.recommend(users=[871,872,873,874,875],k=6)
item_sim_recomm.print_rows(num_rows=70)

+---------+-------------------------------+-------+------+
| user_id |            content            | score | rank |
+---------+-------------------------------+-------+------+
|   871   |  p just wanted to see if l... |  0.0  |  1   |
|   871   |  p hello lucid i've learnt... |  0.0  |  2   |
|   871   | help everyone   \n\ntoday ... |  0.0  |  3   |
|   871   |  p  img src " storage eni ... |  0.0  |  4   |
|   871   |  h1 the beginning  h1 \n p... |  0.0  |  5   |
|   871   | hi guys my name is damian ... |  0.0  |  6   |
|   872   |  p just wanted to see if l... |  0.0  |  1   |
|   872   |  p hello lucid i've learnt... |  0.0  |  2   |
|   872   | help everyone   \n\ntoday ... |  0.0  |  3   |
|   872   |  p  img src " storage eni ... |  0.0  |  4   |
|   872   |  h1 the beginning  h1 \n p... |  0.0  |  5   |
|   872   | hi guys my name is damian ... |  0.0  |  6   |
|   873   |  p just wanted to see if l... |  0.0  |  1   |
|   873   |  p hello lucid i've learnt... |  0.0  |  2  

In [67]:
popularity_model = tc.popularity_recommender.create(train_data_sframe, user_id='user_id', 
                                                       item_id='title', 
                                                       target='actionStrength')

Warning: Ignoring columns content;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 700 observations with 285 users and 546 items.

Data prepared in: 0.741941s

700 observations to process; with 546 unique items.

In [68]:
pop_recomm = popularity_model.recommend(users=[871,872,873,874,875],k=7)
pop_recomm.print_rows(num_rows=70)

+---------+------------------------------+-------+------+
| user_id |            title             | score | rank |
+---------+------------------------------+-------+------+
|   871   |            Am ok             |  3.0  |  1   |
|   871   |         with picture         |  3.0  |  2   |
|   871   |             TEst             |  3.0  |  3   |
|   871   |          jbhjkhkbb           |  3.0  |  4   |
|   871   |     Your face PHP script     |  2.0  |  5   |
|   871   | Lucid post to start.ng Task1 |  1.0  |  6   |
|   871   |       Testing Testing        |  1.0  |  7   |
|   872   |            Am ok             |  3.0  |  1   |
|   872   |         with picture         |  3.0  |  2   |
|   872   |             TEst             |  3.0  |  3   |
|   872   |          jbhjkhkbb           |  3.0  |  4   |
|   872   |     Your face PHP script     |  2.0  |  5   |
|   872   | Lucid post to start.ng Task1 |  1.0  |  6   |
|   872   |       Testing Testing        |  1.0  |  7   |
|   873   |   